In [1]:
def get_database():
    class CustomCSVParser:
        def __init__(self, delimiter=','):
            self.delimiter = delimiter
            self.db = self.MyCustomDB()

        def load(self):
            self._create_tables()
            self._load_all_csvs()
            return self.db

        def _create_tables(self):
            self.db.create_table("zip_code", primary_key="Zip_Code_ID", indexes=["Zip_Code"])
            self.db.create_table("demographics_info", primary_key="Demographics_Info_ID", indexes=["F_Zip_Code_ID"])
            self.db.create_table("inspection_info", primary_key="Inspection_Info_ID", indexes=["F_Restaurant_Info_ID"])
            self.db.create_table("restaurant_info", primary_key="Restaurant_Info_ID", indexes=["F_Zip_Code_ID", "Restaurant_Name", "Categories"])

        def _load_all_csvs(self):
            self.load_csv("../data/zip_code.csv", table_name="zip_code")
            self.load_csv("../data/demographics_info.csv", table_name="demographics_info")
            self.load_csv("../data/inspection_info.csv", table_name="inspection_info")
            self.load_csv("../data/restaurant_info.csv", table_name="restaurant_info")

        def load_csv(self, filepath, table_name):
            import csv
            def infer(value):
                value = value.strip()
                if not value:
                    return None
                for cast in (int, float):
                    try:
                        return cast(value)
                    except ValueError:
                        continue
                return value

            with open(filepath, 'r', encoding='utf-8', newline='') as f:
                reader = csv.reader(f, delimiter=self.delimiter)
                headers = [h.strip() for h in next(reader)]

                for row_values in reader:
                    values = [infer(val) for val in row_values]
                    row = dict(zip(headers, values))
                    self.db.insert(table_name, row)

        class MyCustomDB:
            def __init__(self):
                self.database = {}

            def create_table(self, name, primary_key="id", indexes=None, foreign_keys=None):
                self.database[name] = {
                    "rows": {},
                    "next_id": 1,
                    "primary_key": primary_key,
                    "indexes": {col: {} for col in (indexes or [])},
                    "foreign_keys": foreign_keys or {}
                }

            def insert(self, table_name, row):
                table = self.database[table_name]
                pk = table["primary_key"]

                if pk not in row:
                    row[pk] = table["next_id"]
                    table["next_id"] += 1

                for fk_col, (ref_table, ref_col) in table["foreign_keys"].items():
                    if row[fk_col] not in self.database[ref_table]["indexes"].get(ref_col, {}):
                        raise ValueError(f"Foreign key constraint failed: {fk_col}={row[fk_col]} not found in {ref_table}.{ref_col}")

                key = row[pk]
                table["rows"][key] = row

                for index_col in table["indexes"]:
                    val = row.get(index_col)
                    if val is not None:
                        if val not in table["indexes"][index_col]:
                            table["indexes"][index_col][val] = []
                        table["indexes"][index_col][val].append(key)

            def get_all(self, table_name):
                return list(self.database[table_name]["rows"].values())

            def print_table(self, table_name):
                print(self.database[table_name])
                return

            def print_result_table(self, rows):
                if not rows:
                    print("No data found.")
                    return

                def clean(col): return col.split('.', 1)[-1]

                raw_cols = list(rows[0].keys())
                disp_cols = [clean(c) for c in raw_cols]

                col_widths = {
                    disp_col: max(len(disp_col), max(len(str(row.get(raw_col, ""))) for row in rows))
                    for raw_col, disp_col in zip(raw_cols, disp_cols)
                }

                header = " | ".join(f"{disp_col:<{col_widths[disp_col]}}" for disp_col in disp_cols)
                separator = "-+-".join("-" * col_widths[disp_col] for disp_col in disp_cols)
                print(header)
                print(separator)

                for row in rows:
                    line = " | ".join(
                        f"{str(row.get(raw_col, '')):<{col_widths[clean(raw_col)]}}"
                        for raw_col in raw_cols
                    )
                    print(line)

            def inner_join(self, left_table, right_table, left_key, right_key):
                left_rows = self.get_all(left_table)
                right_rows = self.get_all(right_table)
                joined = []
                for l in left_rows:
                    for r in right_rows:
                        if l.get(left_key) == r.get(right_key):
                            joined.append(
                                {f"{left_table}.{k}": v for k, v in l.items()} |
                                {f"{right_table}.{k}": v for k, v in r.items()}
                            )
                return joined

            def left_join(self, left_table, right_table, left_key, right_key):
                left_rows = self.get_all(left_table)
                right_rows = self.get_all(right_table)
                joined = []
                for l in left_rows:
                    match_found = False
                    for r in right_rows:
                        if l.get(left_key) == r.get(right_key):
                            joined.append(
                                {f"{left_table}.{k}": v for k, v in l.items()} |
                                {f"{right_table}.{k}": v for k, v in r.items()}
                            )
                            match_found = True
                    if not match_found:
                        joined.append(
                            {f"{left_table}.{k}": v for k, v in l.items()} |
                            {f"{right_table}.{k}": None for k in right_rows[0].keys()}
                        )
                return joined

            # ✅ Updated, simple, tuple-based filter
            def select_where_bkup(self, rows, where):
                def match(row, col, op, val):
                    r_val = row.get(col)
                    try:
                        if op == "=": return r_val == val
                        if op == "!=": return r_val != val
                        if op == ">": return r_val is not None and r_val > val
                        if op == "<": return r_val is not None and r_val < val
                        if op == ">=": return r_val is not None and r_val >= val
                        if op == "<=": return r_val is not None and r_val <= val
                        if op == "in": return r_val in val
                        if op == "not in": return r_val not in val
                    except TypeError:
                        return False
                    return False

                # Apply AND logic for all conditions
                return [
                    row for row in rows
                    if all(match(row, col, op, val) for (col, op, val) in where)
                ]

            def select_where(self, table_name, where):
                """
                Unified version of select_where():
                  ✅ Uses primary key or index if available
                  ✅ Supports AND / OR grouped logic
                  ✅ Handles prefixed column names (table.col)
                  ✅ Works for joined temp tables
                  ✅ Case-insensitive for strings
                """
            
                table = self.database[table_name]
                pk = table["primary_key"]
                indexes = table.get("indexes", {})
                rows_by_pk = table["rows"]
            
                # If no WHERE clause → return all rows
                if not where:
                    return list(rows_by_pk.values())
            
                # ------------------------------------------------------------
                # Helper: get value from possibly-prefixed key
                # ------------------------------------------------------------
                def get_value(row, col):
                    if col in row:
                        return row[col]
                    short = col.split(".", 1)[1] if "." in col else col
                    if short in row:
                        return row[short]
                    for k, v in row.items():
                        if k.endswith("." + short):
                            return v
                    return None
            
                # ------------------------------------------------------------
                # Helper: evaluate comparison between one row & condition
                # ------------------------------------------------------------
                def match_row(row, col, op, val):
                    r = get_value(row, col)
                    if isinstance(r, str) and isinstance(val, str):
                        r, val = r.strip().lower(), val.strip().lower()
                    try:
                        if op == "=":      return r == val
                        if op == "!=":     return r != val
                        if op == ">":      return r is not None and r > val
                        if op == "<":      return r is not None and r < val
                        if op == ">=":     return r is not None and r >= val
                        if op == "<=":     return r is not None and r <= val
                        if op == "in":     return r in val
                        if op == "not in": return r not in val
                    except Exception:
                        return False
                    return False
            
                # ------------------------------------------------------------
                # Step 1: Try index / primary-key optimization for first condition
                # ------------------------------------------------------------
                candidate_keys = None
                first_group = where[0]
                if first_group and isinstance(first_group[0], tuple):
                    first_col, first_op, first_val = first_group[0]
                    first_col_short = first_col.split(".", 1)[1] if "." in first_col else first_col
            
                    if first_op == "=":
                        # Primary key lookup
                        if first_col_short == pk and first_val in rows_by_pk:
                            candidate_keys = [first_val]
                        # Index lookup
                        elif first_col_short in indexes:
                            candidate_keys = indexes[first_col_short].get(first_val, [])
                    elif first_op == "in" and isinstance(first_val, (list, tuple, set)):
                        if first_col_short in indexes:
                            candidate_keys = []
                            for v in first_val:
                                candidate_keys += indexes[first_col_short].get(v, [])
            
                # ------------------------------------------------------------
                # Step 2: Build initial candidate rows
                # ------------------------------------------------------------
                if candidate_keys is not None:
                    candidate_rows = [rows_by_pk[k] for k in candidate_keys if k in rows_by_pk]
                else:
                    candidate_rows = list(rows_by_pk.values())
            
                # ------------------------------------------------------------
                # Step 3: Evaluate grouped AND/OR logic for each candidate row
                # ------------------------------------------------------------
                filtered = []
            
                def evaluate_group(row, group):
                    if not group:
                        return True
                    logic = "AND"
                    results = []
                    for cond in group:
                        if isinstance(cond, str) and cond.upper() in {"AND", "OR"}:
                            logic = cond.upper()
                            continue
                        col, op, val = cond
                        results.append(match_row(row, col, op, val))
                    return (all(results) if logic == "AND" else any(results)), logic
            
                for row in candidate_rows:
                    group_results, connectors = [], []
                    for group in where:
                        g_res, g_logic = evaluate_group(row, group)
                        group_results.append(g_res)
                        # connector between groups
                        last = group[-1]
                        connectors.append(last.upper() if isinstance(last, str) and last.upper() in {"AND", "OR"} else "OR")
            
                    # combine group results sequentially
                    res = group_results[0]
                    for i in range(1, len(group_results)):
                        conn = connectors[i - 1]
                        res = (res and group_results[i]) if conn == "AND" else (res or group_results[i])
            
                    if res:
                        filtered.append(row)
            
                # ------------------------------------------------------------
                # Step 4: Return filtered list
                # ------------------------------------------------------------
                return filtered







            
            def group_by(self, rows, group_key, agg_col, agg_fn):
                from collections import defaultdict

                func_map = {
                    "avg": lambda vals: round(sum(vals) / len(vals), 2) if vals else None,
                    "sum": lambda vals: sum(vals) if vals else 0,
                    "count": lambda vals: len(vals),
                    "max": lambda vals: max(vals) if vals else None,
                    "min": lambda vals: min(vals) if vals else None
                }
                if agg_fn not in func_map:
                    valid = ", ".join(func_map.keys())
                    print(f"Invalid agg_fn: {agg_fn}. Valid options are: {valid}")
                    return []
                    #raise ValueError(f"Invalid agg_fn: {agg_fn}. Valid options are: {valid}")
            
                agg_func = func_map[agg_fn]
                
                if group_key:
                    grouped = defaultdict(list)
                    for row in rows:
                        grouped[row.get(group_key)].append(row.get(agg_col))
                    table, col = agg_col.split(".", 1)
                    return [
                        {
                            group_key: k,
                            f"{table}.{agg_fn}_{col}": agg_func(
                                [v for v in vals if isinstance(v, (int, float))]
                            )
                        }
                        for k, vals in grouped.items()
                    ]
                else:
                    table, col = agg_col.split(".", 1)
                    #print(table, col)
                    #print(rows)
                    #for row in rows:
                    #    print(row.get(agg_col))
                    values = [row.get(agg_col) for row in rows if isinstance(row.get(agg_col), (int, float))]
                    return [{f"{table}.{agg_fn}_{col}": agg_func(values)}]

            
            def project_columns(self, rows, select):
                result = []
                for row in rows:
                    new_row = {}
                    for col in select:
                        # Try exact match first
                        if col in row:
                            new_row[col] = row[col]
                        else:
                            # Try without prefix if it's missing
                            short_col = col.split('.')[-1]
                            match = next((v for k, v in row.items() if k.endswith(short_col)), None)
                            new_row[col] = match
                    result.append(new_row)
                return result

             
                def sort_key(row):
                    key = []
                    for col, desc in zip(order_by, descending):
                        val = row.get(col)
                        if isinstance(val, (int, float)) and desc:
                            key.append(-val if val is not None else float('inf'))
                        else:
                            # For descending on strings, invert with a tuple
                            key.append((val is None, val if not desc else _reverse_str(val)))
                    return tuple(key)
                
                def _reverse_str(val):
                    if isinstance(val, str):
                        return ''.join(chr(255 - ord(c)) for c in val)
                    return val
                
                return sorted(rows, key=sort_key)

            def order_by_rows(self, rows, order_by, descending=False):
                if not rows:
                    return rows
            
                # Normalize inputs
                if isinstance(order_by, str):
                    order_by = [order_by]
                if isinstance(descending, bool):
                    descending = [descending] * len(order_by)
            
                # Apply sorts in reverse order (to preserve earlier priorities)
                for col, desc in reversed(list(zip(order_by, descending))):
                    rows.sort(
                        key=lambda r: (r.get(col) is None, r.get(col)),
                        reverse=desc
                    )
                return rows




            def select_query(self,
                             from_table,
                             joins=None,
                             where=None,
                             group_by=None,
                             agg_col=None,
                             agg_fn=None,
                             columns=None,
                             order_by=None,
                             descending=False):
                rows = list(self.database[from_table]["rows"].values())
                rows = [{f"{from_table}.{k}": v for k, v in row.items()} for row in rows]
                base = from_table

                # Apply joins
                if joins:
                    for join_table, on_keys, join_type in joins:
                        left_key, right_key = on_keys
                        if join_type == "inner":
                            rows = self.inner_join(base, join_table, left_key, right_key)
                        elif join_type == "left":
                            rows = self.left_join(base, join_table, left_key, right_key)
                        else:
                            raise ValueError("Only 'inner' and 'left' joins are supported.")
                        base = "tmpTable"
                        self.create_table(base, primary_key="id")
                        for i, row in enumerate(rows):
                            row_with_id = {"id": i + 1}
                            row_with_id.update(row)
                            self.insert(base, row_with_id)

                # Apply WHERE
                # Apply WHERE
                if where:
                    rows = self.select_where(base, where)   # ← use current base table (tmpTable after join)
                    # Re-apply prefixes only if the base is the original table (when no join)
                    if base == from_table:
                        rows = [{f"{from_table}.{k}": v for k, v in row.items()} for row in rows]


                # Apply aggregation if specified or not
                if agg_col and agg_fn:
                    rows = self.group_by(rows, group_by, agg_col, agg_fn)
               
                # Apply column selection
                if columns:
                    rows = self.project_columns(rows, columns)
                if order_by:
                    rows = self.order_by_rows(rows, order_by, descending)
                # Cleanup temporary table
                self.database.pop("tmpTable", None)
                return rows

    return CustomCSVParser().load()
MyDataFrame = get_database()

In [2]:
result = MyDataFrame.select_query(
    from_table="restaurant_info",
    joins=[("inspection_info", ("Restaurant_Info_ID", "F_Restaurant_Info_ID"), "inner")],
    where=[
        [("restaurant_info.Categories", "=", "Mexican"),
         ("restaurant_info.Review_Count", ">", 100), "AND"]
    ],
    columns=[
        "restaurant_info.Restaurant_Name",
        "restaurant_info.Categories",
        "restaurant_info.Review_Count",
        "inspection_info.Score"
    ]
)
db.print_result_table(result)


Restaurant_Name                        | Categories | Review_Count | Score
---------------------------------------+------------+--------------+------
El Senor Taco                          | Mexican    | 103          | 97.0 
Taqueria Tijuana                       | Mexican    | 156          | 96.0 
La Carreta                             | Mexican    | 227          | 93.0 
El Taco Mexicano No 2                  | Mexican    | 105          | 91.0 
Chicharroland                          | Mexican    | 138          | 92.0 
Tacos Gavilan                          | Mexican    | 276          | 95.0 
Los Cinco Puntos                       | Mexican    | 920          | 95.0 
El Patroncito Mexican Cuisine          | Mexican    | 294          | 96.0 
Mariscos Linda                         | Mexican    | 228          | 94.0 
Las Islitas No 1                       | Mexican    | 104          | 91.0 
Somos Oaxaca                           | Mexican    | 178          | 95.0 
Tacos & Mariscos El Parqu

In [3]:
result = db.select_query(
    from_table="restaurant_info",
    joins=[("inspection_info", ("Restaurant_Info_ID", "F_Restaurant_Info_ID"), "inner")],
    where=[
        [("restaurant_info.Categories", "=", "Fast Food"),
         ("restaurant_info.Categories", "=", "Pizza"), "OR"],
    ],
    columns=[
        "restaurant_info.Restaurant_Name",
        "restaurant_info.Categories",
        "restaurant_info.Review_Count",
        "inspection_info.Score"
    ]
)
db.print_result_table(result)

Restaurant_Name                      | Categories | Review_Count | Score
-------------------------------------+------------+--------------+------
Bangin Buns                          | Fast Food  | 110          | 93.0 
Pioneer Chicken                      | Fast Food  | 564          | 95.0 
Tumby's Pizza                        | Pizza      | 65           | 97.0 
Pizzamania                           | Pizza      | 77           | 95.0 
Golden Ox                            | Fast Food  | 62           | 94.0 
Domino's Pizza                       | Pizza      | 43           | 97.0 
Piara Pizza                          | Pizza      | 22           | 92.0 
Pizza King                           | Pizza      | 33           | 94.0 
Amanecer Ylobasqence Y Familia       | Fast Food  | 13           | 92.0 
Louisiana Fried Chicken              | Fast Food  | 12           | 91.0 
Pizzeria Mozza                       | Pizza      | 4730         | 98.0 
Masa of Echo Park                    | Pizza      |

In [4]:
result = db.select_query(
    from_table="restaurant_info",
    joins=[("inspection_info", ("Restaurant_Info_ID", "F_Restaurant_Info_ID"), "inner")],
    where=[
        [("restaurant_info.Categories", "=", "Fast Food"),
         ("restaurant_info.Review_Count", ">", 100), "AND"],
        [("restaurant_info.Categories", "=", "Pizza"), "OR"]
    ],
    columns=[
        "restaurant_info.Restaurant_Name",
        "restaurant_info.Categories",
        "restaurant_info.Review_Count",
        "inspection_info.Score"
    ]
)
db.print_result_table(result)


No data found.


In [5]:
result = db.select_query(
    from_table="restaurant_info",
    joins=[("inspection_info", ("Restaurant_Info_ID", "F_Restaurant_Info_ID"), "inner")],
    where=[
        [("restaurant_info.Categories", "=", "Fast Food"),],
    ],
    columns=[
        "restaurant_info.Restaurant_Name",
        "restaurant_info.Categories",
        "restaurant_info.Review_Count",
        "inspection_info.Score"
    ]
)
db.print_result_table(result)


Restaurant_Name                | Categories | Review_Count | Score
-------------------------------+------------+--------------+------
Bangin Buns                    | Fast Food  | 110          | 93.0 
Pioneer Chicken                | Fast Food  | 564          | 95.0 
Golden Ox                      | Fast Food  | 62           | 94.0 
Amanecer Ylobasqence Y Familia | Fast Food  | 13           | 92.0 
Louisiana Fried Chicken        | Fast Food  | 12           | 91.0 
Cafe El Colibri                | Fast Food  | 1            | 90.0 
SKECHERS Food Spot             | Fast Food  | 273          | 97.0 
Basil Thai Kitchen             | Fast Food  | 18           | 97.0 
Fry Shack                      | Fast Food  | 8            | 96.0 


In [6]:
result = db.select_query(
    from_table="demographics_info",
    agg_col="demographics_info.Total_Population",
    agg_fn="max",  # or min
    columns=["demographics_info.max_Total_Population"]
)
for row in result:
    print(row)
db.print_result_table(result)

{'demographics_info.max_Total_Population': 109511}
max_Total_Population
--------------------
109511              


In [7]:

# Need to test this
result = db.select_query(
    from_table="restaurant_info",
    joins=[("inspection_info", ("Restaurant_Info_ID", "F_Restaurant_Info_ID"), "inner")],
    agg_col="inspection_info.Score",
    agg_fn="avg",
    columns=["inspection_info.avg_Score"]
)
for row in result:
    print(row)
db.print_result_table(result)

{'inspection_info.avg_Score': 94.08}
avg_Score
---------
94.08    


In [8]:
db = get_database()
# Non-grouped SUM
result = db.select_query(
    from_table="demographics_info",
    agg_col="demographics_info.Total_Population",
    agg_fn="sum",
    columns=["demographics_info.sum_Total_Population"]
)

for row in result:
    print(row)
db.print_result_table(result)

{'demographics_info.sum_Total_Population': 6388999}
sum_Total_Population
--------------------
6388999             


In [9]:

result = db.select_query(
    from_table="restaurant_info",
    columns=["restaurant_info.Restaurant_Name","restaurant_info.Categories",  "restaurant_info.Review_Count"],
    order_by=["restaurant_info.Restaurant_Name", "restaurant_info.Review_Count"],
    descending=[False, True]
)
db.print_result_table(result)

Restaurant_Name                                        | Categories                                                    | Review_Count
-------------------------------------------------------+---------------------------------------------------------------+-------------
& Waffles                                              | Breakfast & Brunch, Waffles, Sandwiches                       | 2163        
& Waffles                                              | Breakfast & Brunch, Waffles, Burgers                          | 1100        
1 Star Donut                                           | Donuts, Sandwiches                                            | 22          
101 Asian Kitchen                                      | Chinese, Sushi Bars, Hawaiian                                 | 274         
2 For 1 Pizza                                          | Pizza                                                         | 194         
23rd Street Cafe                                       | India

In [10]:
db = get_database()
result = db.select_query(
    from_table="restaurant_info",
    where=[
        [("restaurant_info.Categories", "=", "Mexican"),]
    ],
    columns=["restaurant_info.Restaurant_Name", "restaurant_info.Categories", "restaurant_info.Review_Count"]
)
db.print_result_table(result)


Restaurant_Name                        | Categories | Review_Count
---------------------------------------+------------+-------------
El Senor Taco                          | Mexican    | 103         
Taqueria Tijuana                       | Mexican    | 156         
La Carreta                             | Mexican    | 227         
El Tacorin                             | Mexican    | 86          
El Taco Mexicano No 2                  | Mexican    | 105         
Chicharroland                          | Mexican    | 138         
Tacos Gavilan                          | Mexican    | 276         
Los Cinco Puntos                       | Mexican    | 920         
El Patroncito Mexican Cuisine          | Mexican    | 294         
Mariscos Linda                         | Mexican    | 228         
Grandma's Kitchen Kaffe                | Mexican    | 55          
Tacos Los Carnales                     | Mexican    | 78          
El Taco Rapido                         | Mexican    | 87      

In [11]:
result = db.select_query(
    from_table="restaurant_info",
    columns=["restaurant_info.Restaurant_Name", "restaurant_info.Categories", "restaurant_info.Review_Count"],
    order_by=["restaurant_info.Review_Count","restaurant_info.Restaurant_Name"],
    descending=[True, True]
)
db.print_result_table(result)

Restaurant_Name                                        | Categories                                                    | Review_Count
-------------------------------------------------------+---------------------------------------------------------------+-------------
Bottega Louie                                          | Italian, Desserts, Breakfast & Brunch                         | 19291       
Howlin' Ray's                                          | Southern, Chicken Shop, American                              | 7974        
Republique                                             | French, Breakfast & Brunch, Cocktail Bars                     | 7619        
Philippe                                               | Sandwiches, Breakfast & Brunch, American                      | 7495        
The Griddle Cafe                                       | Breakfast & Brunch, Coffee & Tea, Cocktail Bars               | 6865        
Eggslut                                                | Break

In [12]:

db = get_database()
#db.print_table("restaurant_info")
result = db.select_query(
    from_table="restaurant_info",
    agg_col="restaurant_info.Rating",
    agg_fn="sum",
    columns=["restaurant_info.sum_Rating"]
)
for row in result:
    print(row)
db.print_result_table(result)

{'restaurant_info.sum_Rating': 19081.4}
sum_Rating
----------
19081.4   


In [13]:
#db.print_table("restaurant_info")
db = get_database()
result = db.select_query(
    from_table="restaurant_info",
    joins=[("inspection_info", ("Restaurant_Info_ID", "F_Restaurant_Info_ID"), "inner")],
    group_by="restaurant_info.Categories",
    agg_col="inspection_info.Score",
    agg_fn="avg",
    columns=["restaurant_info.Categories","inspection_info.avg_Score"]
)
#db.print_table("inspection_info")
db.print_result_table(result)
for row in result:
    print(row)

Categories                                                    | avg_Score
--------------------------------------------------------------+----------
Mexican                                                       | 93.62    
Mexican, Breakfast & Brunch, Salvadoran                       | 93.5     
Fast Food                                                     | 93.89    
Food Trucks, American, Comfort Food                           | 92.0     
Burgers                                                       | 93.79    
Sushi Bars                                                    | 93.54    
Seafood, Cajun/Creole                                         | 95.25    
Soul Food, Southern                                           | 93.0     
Southern, Breakfast & Brunch, Cocktail Bars                   | 96.0     
Mexican, Seafood                                              | 92.78    
Sports Bars, Mexican, Cocktail Bars                           | 91.0     
Empanadas, Food Delivery Services, Arg

In [14]:

db = get_database()
# Non-grouped SUM
result = db.select_query(
    from_table="demographics_info",
    agg_col="demographics_info.Total_Population",
    agg_fn="sum",
    columns=["demographics_info.sum_Total_Population"]
)

for row in result:
    print(row)
db.print_result_table(result)

{'demographics_info.sum_Total_Population': 6388999}
sum_Total_Population
--------------------
6388999             


In [15]:
result = db.select_query(
    from_table="demographics_info",
    agg_col="demographics_info.Total_Population",
    agg_fn="sum",
    columns=["demographics_info.sum_Total_Population"]
)# SUM


result = db.select_query(
    from_table="demographics_info",
    group_by="demographics_info.F_Zip_Code_ID",
    agg_col="demographics_info.Total_Population",
    agg_fn="sum",
    columns=["demographics_info.F_Zip_Code_ID", "demographics_info.sum_Total_Population"]
)



# AVG
result = db.select_query(
    from_table="restaurant_info",
    joins=[("inspection_info", ("Restaurant_Info_ID", "F_Restaurant_Info_ID"), "inner")],
    group_by="restaurant_info.Restaurant_Name",
    agg_col="inspection_info.Score",
    agg_fn="avg",
    columns=["restaurant_info.Restaurant_Name", "inspection_info.avg_Score"]
)
for row in result:
    print(row)
db.print_result_table(result)
# Need to test this
result = db.select_query(
    from_table="restaurant_info",
    joins=[("inspection_info", ("Restaurant_Info_ID", "F_Restaurant_Info_ID"), "inner")],
    agg_col="inspection_info.Score",
    agg_fn="avg",
    columns=["inspection_info.avg_Score"]
)


# COUNT
result = db.select_query(
    from_table="restaurant_info",
    agg_col="restaurant_info.Rating",
    agg_fn="count",
    columns=["restaurant_info.count_Rating"]
)

# MIN / MAX
result = db.select_query(
    from_table="demographics_info",
    group_by="demographics_info.F_Zip_Code_ID",
    agg_col="demographics_info.Population",
    agg_fn="min",  # or min
    columns=["demographics_info.F_Zip_Code_ID", "demographics_info.min_Total_Population"]
)



{'restaurant_info.Restaurant_Name': 'El Senor Taco', 'inspection_info.avg_Score': 97.0}
{'restaurant_info.Restaurant_Name': 'Mi Pueblo Salvadorian Restaurant #1', 'inspection_info.avg_Score': 92.0}
{'restaurant_info.Restaurant_Name': 'Bangin Buns', 'inspection_info.avg_Score': 93.0}
{'restaurant_info.Restaurant_Name': 'Taqueria Tijuana', 'inspection_info.avg_Score': 96.0}
{'restaurant_info.Restaurant_Name': 'The Lobos Truck', 'inspection_info.avg_Score': 92.0}
{'restaurant_info.Restaurant_Name': 'Boys Burgers', 'inspection_info.avg_Score': 95.0}
{'restaurant_info.Restaurant_Name': 'La Zona Sushi', 'inspection_info.avg_Score': 94.0}
{'restaurant_info.Restaurant_Name': 'L.A. Crazy Crab', 'inspection_info.avg_Score': 96.0}
{'restaurant_info.Restaurant_Name': "Granny's Kitchen", 'inspection_info.avg_Score': 93.0}
{'restaurant_info.Restaurant_Name': 'Delicious at The Dunbar', 'inspection_info.avg_Score': 96.0}
{'restaurant_info.Restaurant_Name': 'Mariscos Mi Lindo Nayarit', 'inspection_info

In [20]:
result = db.select_query(
    from_table="restaurant_info",
    joins=[("inspection_info", ("Restaurant_Info_ID", "F_Restaurant_Info_ID"), "inner")],
    where=[
        [("restaurant_info.Categories", "=", "Fast Food"),
         ("restaurant_info.Review_Count", ">", 100), "AND"]
    ],
    columns=[
        "restaurant_info.Restaurant_Name",
        "restaurant_info.Review_Count",
        "inspection_info.Score"
    ]
)
db.print_result_table(result)


Restaurant_Name    | Review_Count | Score
-------------------+--------------+------
Bangin Buns        | 110          | 93.0 
Pioneer Chicken    | 564          | 95.0 
SKECHERS Food Spot | 273          | 97.0 


In [19]:
result = db.select_query(
    from_table="restaurant_info",
    joins=[("inspection_info", ("Restaurant_Info_ID", "F_Restaurant_Info_ID"), "inner")],
    where=[
        [("restaurant_info.Categories", "=", "Fast Food")]
    ],
    columns=[
        "restaurant_info.Restaurant_Name",
        "restaurant_info.Created_At",
        "inspection_info.Score",
        "restaurant_info.Review_Count",
        "inspection_info.Created_At"
    ]
)
db.print_result_table(result)


Restaurant_Name                | Created_At          | Score | Review_Count | Created_At         
-------------------------------+---------------------+-------+--------------+--------------------
Bangin Buns                    | 2025-04-13 14:10:08 | 93.0  | 110          | 2025-04-14 00:50:50
Pioneer Chicken                | 2025-04-13 14:10:08 | 95.0  | 564          | 2025-04-14 02:11:44
Golden Ox                      | 2025-04-13 14:10:08 | 94.0  | 62           | 2025-04-14 02:54:45
Amanecer Ylobasqence Y Familia | 2025-04-13 14:10:08 | 92.0  | 13           | 2025-04-14 21:56:56
Louisiana Fried Chicken        | 2025-04-13 14:10:08 | 91.0  | 12           | 2025-04-14 18:21:39
Cafe El Colibri                | 2025-04-13 14:10:08 | 90.0  | 1            | 2025-04-14 20:28:41
SKECHERS Food Spot             | 2025-04-13 14:10:08 | 97.0  | 273          | 2025-04-16 04:40:56
Basil Thai Kitchen             | 2025-04-13 14:10:08 | 97.0  | 18           | 2025-04-16 07:10:34
Fry Shack           